<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/nlp_project/milestone_project_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lets reproduce a research paper and try to increase its accuracy**

PubMed 200k RCT: a Dataset for Sequential Sentence Classification in Medical Abstracts  [https://arxiv.org/abs/1710.06071]

# Dataset Used in Research Paper
[https://github.com/Franck-Dernoncourt/pubmed-rct]